# Introduction to NLP to SQL Test Cases with GPT-4

In this Jupyter Notebook, we will explore how GPT-4, a state-of-the-art language model by OpenAI, can be used to create NLP to SQL test cases and convert natural language questions into SQL queries with ease. GPT-4 has shown significant improvements over its predecessor, GPT-3.5, in understanding context and generating more accurate and complex responses. Although GPT-3.5 is less advanced, it still demonstrates respectable performance in various NLP tasks, including generating SQL queries from natural language inputs.

By the end of this notebook, we will demonstrate the capabilities of both GPT-4 and GPT-3.5 in handling NLP to SQL conversion tasks, showcasing the advancements in the field of natural language processing and their potential applications in real-world scenarios.


In [1]:
import openai
import json
import os
import pandas as pd
import sqlite3
import numpy as np

# Params
openai.api_key = os.getenv("OPENAI_API_KEY")
pd.set_option('display.max_colwidth', None)

#### A. Create SQL database with the EBA risk data

In [2]:
# Read the CSV file into a pandas DataFrame
test_cases = pd.read_csv('gpt_generated_test_cases.txt', sep='|')

# Show dataset
test_cases.sample(n=5)

,test_number,user_query,corresponding_sql_query,difficulty_level
11,12,How many records are there for each segment?,"SELECT Segment, COUNT(*) FROM eba_data GROUP BY Segment;",Easy
10,11,Which date has the lowest observed_default_rate for Portugal?,SELECT Date FROM eba_data WHERE Country = 'Portugal' AND observed_default_rate = (SELECT MIN(observed_default_rate) FROM eba_data WHERE Country = 'Portugal');,Hard
12,13,What is the total observed_loss_rate for all countries in the Corporates - Of Which: SME segment?,SELECT SUM(observed_loss_rate) FROM eba_data WHERE Segment = 'Corporates - Of Which: SME';,Medium
3,4,Give me the maximum observed_loss_rate for all countries,SELECT MAX(observed_loss_rate) FROM eba_data;,Medium
7,8,Find the average observed_loss_rate for Corporates segment in all EU countries,SELECT AVG(observed_loss_rate) FROM eba_data WHERE Segment = 'Corporates';,Medium


In [3]:
# Read the CSV file into a pandas DataFrame
eba_data = pd.read_csv('db_eba_data''.csv', sep=',')
eba_data['Date'] = pd.to_datetime(eba_data['Date'], format="%b-%y")

# Write the dataFrame to the SQLite database
with sqlite3.connect('eba.db') as conn:
    eba_data.to_sql('eba_data', conn, if_exists='replace', index=False)

# Show dataset
eba_data.sample(n=5)

,Country,Segment,Date,observed_default_rate,observed_loss_rate,estimated_pd,estimated_lgd
944,Luxembourg,Retail - Other Retail,2021-06-01,1.24%,15.44%,1.98%,22.38%
620,Germany,Retail - Secured on real estate property,2021-09-01,0.38%,8.66%,0.72%,18.17%
1821,Spain,Corporates - Of Which: Specialised Lending,2020-09-01,1.90%,9.25%,1.41%,21.92%
905,Hungary,Corporates - Of Which: SME,2021-06-01,3.00%,50.48%,2.74%,46.17%
2057,Norway,Retail - Other Retail,2020-06-01,0.66%,30.83%,2.12%,31.37%


In [4]:
eba_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Country                2457 non-null   object        
 1   Segment                2457 non-null   object        
 2   Date                   2457 non-null   datetime64[ns]
 3   observed_default_rate  2010 non-null   object        
 4   observed_loss_rate     1693 non-null   object        
 5   estimated_pd           2436 non-null   object        
 6   estimated_lgd          2436 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 135.0+ KB


#### B. Test GPT generated sql statements against a real dataset

In [5]:
def query_database(db, openai_query):
    conn = sqlite3.connect(db)

    # execute sql query and retrieve data as pandas dataframe
    df = pd.read_sql_query(openai_query, conn, parse_dates=['Date'])

    # close database connection
    conn.close()
    
    return df


# Iterate over the rows in the DataFrame using iterrows()
for index, row in test_cases.iterrows():
    query = row['corresponding_sql_query']
    
    try:
        df_result = query_database('eba.db', query)
        if len(df_result) > 0:
            print(f"Success: test case {index}, rows returned: {len(df_result)}, difficulty: {row['difficulty_level']}")
        else:
            print(f"Failure: test case {index}, difficulty: {row['difficulty_level']}")
        
    except Exception as e:
        print(f"Failure: test case {index}, difficulty: {row['difficulty_level']}: {e}")

Success: test case 0, rows returned: 63, difficulty: Easy
Success: test case 1, rows returned: 63, difficulty: Easy
Success: test case 2, rows returned: 1, difficulty: Medium
Success: test case 3, rows returned: 1, difficulty: Medium
Success: test case 4, rows returned: 9, difficulty: Medium
Success: test case 5, rows returned: 1, difficulty: Hard
Success: test case 6, rows returned: 40, difficulty: Hard
Success: test case 7, rows returned: 1, difficulty: Medium
Success: test case 8, rows returned: 59, difficulty: Medium
Success: test case 9, rows returned: 9, difficulty: Hard
Success: test case 10, rows returned: 1, difficulty: Hard
Success: test case 11, rows returned: 8, difficulty: Easy
Success: test case 12, rows returned: 1, difficulty: Medium
Failure: test case 13, difficulty: Very Hard
Failure: test case 14, difficulty: Very Hard: Execution failed on sql 'SELECT CORR(observed_default_rate, estimated_pd) FROM eba_data WHERE Segment = 'Retail - Qualifying Revolving';': no such fu

#### C. Test if GPT 3.5 returns meaningful SQL queries

In [6]:
def segment_openai(user_query):
    """ Take a user query and send it to ChatGPT"""
    completion = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages=[
                                    {"role": "system", "content": "You are an SQL expert."},
                                    {"role": "user", "content": "I have an SQLite table 'eba_data' with columns Country, Segment, Date, observed_default_rate, observed_loss_rate, estimated_pd, estimated_lgd. The valid values for the 'Country' column are all European Union countries. The valid values for the 'Segment' column are: 'Corporates', 'Corporates - Of Which: Specialised Lending', 'Corporates - Of Which: SME', 'Retail', 'Retail - Secured on real estate property', 'Retail - Qualifying Revolving', 'Retail - Other Retail', 'Retail - Secured on real estate property - Of Which: non-SME', 'Retail - Secured on real estate property - Of Which: SME', 'Retail - Other Retail - Of Which: non-SME', 'Retail - Other Retail - Of Which: SME'. 4 last columns are in the float format."},
                                    {"role": "user", "content": f"Return plain syntactically correct SQLite query given the user input: {user_query}"}
                                ]
                        )
    
    return completion.choices[0].message.content, completion

In [7]:
# Iterate over the rows in the DataFrame using iterrows()
for index, row in test_cases.sample(4).iterrows():
    gpt_user_query = row['user_query']
    print(f'User query: {gpt_user_query} \n')
    
    sql_query, _ = segment_openai(gpt_user_query)
    print(f'GPT proposal: {sql_query}\n')

User query: What is the total observed_loss_rate for all countries in the Corporates - Of Which: SME segment? 

GPT proposal: SELECT SUM(observed_loss_rate) FROM eba_data WHERE Segment='Corporates - Of Which: SME';

User query: Find the total observed_loss_rate for each country in Retail - Secured on real estate property - Of Which: non-SME segment, where estimated_lgd is greater than 0.5 

GPT proposal: Here is the requested SQLite query:

```
SELECT Country, SUM(observed_loss_rate) AS total_loss_rate
FROM eba_data
WHERE Segment = 'Retail - Secured on real estate property - Of Which: non-SME' AND estimated_lgd > 0.5 
GROUP BY Country;
``` 

This query uses the `SUM` and `GROUP BY` clauses to calculate the total observed_loss_rate for each country in the specified segment where estimated_lgd is greater than 0.5.

User query: Show me the data for Germany 

GPT proposal: SELECT * FROM eba_data WHERE Country = 'Germany';

User query: List the data for Retail segment in France 

GPT propos

#### D. Test if GPT 4 returns meaningful SQL queries

Prompt:
- I have an SQLite table 'eba_data' with columns Country, Segment, Date, observed_default_rate, observed_loss_rate, estimated_pd, estimated_lgd. The valid values for the 'Country' column are all European Union countries. The valid values for the 'Segment' column are: 'Corporates', 'Corporates - Of Which: Specialised Lending', 'Corporates - Of Which: SME', 'Retail', 'Retail - Secured on real estate property', 'Retail - Qualifying Revolving', 'Retail - Other Retail', 'Retail - Secured on real estate property - Of Which: non-SME', 'Retail - Secured on real estate property - Of Which: SME', 'Retail - Other Retail - Of Which: non-SME', 'Retail - Other Retail - Of Which: SME'. 4 last columns are in the float format.
- Return plain syntactically correct SQLite query given the user input: "Give me the maximum observed_loss_rate for all countries"

In [8]:
query = """
        SELECT Country, MAX(observed_loss_rate) AS max_observed_loss_rate
        FROM eba_data
        GROUP BY Country;
"""

query_database('eba.db', query)

,Country,max_observed_loss_rate
0,None,None
1,Australia,9.10%
2,Austria,8.55%
3,Belgium,9.80%
4,Bulgaria,95.67%
5,Canada,8.53%
6,China,9.61%
7,Croatia,61.98%
8,Cyprus,67.18%
9,Czech,82.02%


Prompt:
- I have an SQLite table 'eba_data' with columns Country, Segment, Date, observed_default_rate, observed_loss_rate, estimated_pd, estimated_lgd. The valid values for the 'Country' column are all European Union countries. The valid values for the 'Segment' column are: 'Corporates', 'Corporates - Of Which: Specialised Lending', 'Corporates - Of Which: SME', 'Retail', 'Retail - Secured on real estate property', 'Retail - Qualifying Revolving', 'Retail - Other Retail', 'Retail - Secured on real estate property - Of Which: non-SME', 'Retail - Secured on real estate property - Of Which: SME', 'Retail - Other Retail - Of Which: non-SME', 'Retail - Other Retail - Of Which: SME'. 4 last columns are in the float format.
- Return plain syntactically correct SQLite query given the user input: "Calculate the percentage change in average estimated_lgd between the first half and second half of the year 2022 for each country"

In [9]:
query = """
        WITH
          first_half AS (
            SELECT
              Country,
              AVG(estimated_lgd) AS avg_estimated_lgd_first_half
            FROM eba_data
            WHERE Date BETWEEN '2021-01-01' AND '2021-06-30'
            GROUP BY Country
          ),
          second_half AS (
            SELECT
              Country,
              AVG(estimated_lgd) AS avg_estimated_lgd_second_half
            FROM eba_data
            WHERE Date BETWEEN '2021-07-01' AND '2021-12-31'
            GROUP BY Country
          )
        SELECT
          f.Country,
          ((s.avg_estimated_lgd_second_half - f.avg_estimated_lgd_first_half) / f.avg_estimated_lgd_first_half) * 100 AS percentage_change
        FROM first_half f
        JOIN second_half s ON f.Country = s.Country
        ORDER BY f.Country;

"""

query_database('eba.db', query)

,Country,percentage_change
0,Australia,-2.453495
1,Austria,13.167481
2,Belgium,-0.266430
3,Bulgaria,-18.031419
4,Canada,-1.055848
5,China,0.687293
6,Croatia,4.458599
7,Cyprus,-5.156058
8,Czech,-0.854990
9,Denmark,1.548403
